Our task is to get rid of as much data as possible without dropping below an average of 90% accuracy in a 10-fold cross validation. We will then see if we can recreate the experiment without using payment amount, outstanding principals, or anything related.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
pd.set_option('display.height', 1000)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [2]:
y2015 = pd.read_csv(
    'LoanStats3d.csv',
    skipinitialspace=True,
    header=1
)

# Note the warning about dtypes.

/Users/c/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (0,19,55) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
y2015.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_il_6m,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
0,68009401,72868139.0,16000.0,16000.0,16000.0,60 months,14.85%,379.39,C,C5,Bookkeeper/Accounting,10+ years,MORTGAGE,48000.0,Not Verified,Dec-2015,Current,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,credit_card,Credit card refinancing,297xx,SC,33.18,0.0,Jun-1991,0.0,33.0,2.0,11.0,2.0,19108.0,29.6%,19.0,w,13668.88,13668.88,4519.68,4519.68,2331.12,2188.56,0.0,0.0,0.0,Jan-2017,379.39,Jan-2017,Jan-2017,0.0,33.0,1.0,INDIVIDUAL,NaN,NaN,NaN,0.0,0.0,31329.0,1.0,2.0,1.0,2.0,6.0,12221.0,91.0,1.0,4.0,14982.0,65.0,284700.0,0.0,0.0,1.0,6.0,2848.0,263953.0,17.6,0.0,0.0,131.0,294.0,11.0,6.0,2.0,11.0,76.0,11.0,76.0,2.0,6.0,9.0,6.0,8.0,6.0,9.0,11.0,9.0,11.0,0.0,0.0,0.0,2.0,78.9,0.0,0.0,2.0,298100.0,31329.0,281300.0,13400.0
1,68354783,73244544.0,9600.0,9600.0,9600.0,36 months,7.49%,298.58,A,A4,tech,8 years,MORTGAGE,60000.0,Not Verified,Dec-2015,Current,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,credit_card,Credit card refinancing,299xx,SC,22.44,0.0,Jun-1996,0.0,NaN,NaN,7.0,0.0,7722.0,59.4%,9.0,w,6635.69,6635.69,3572.97,3572.97,2964.31,608.66,0.0,0.0,0.0,Jan-2017,298.58,Jan-2017,Jan-2017,0.0,NaN,1.0,INDIVIDUAL,NaN,NaN,NaN,0.0,0.0,55387.0,0.0,3.0,0.0,0.0,47.0,47665.0,43.0,2.0,2.0,4744.0,50.0,13000.0,0.0,0.0,2.0,2.0,7912.0,4778.0,61.8,0.0,0.0,234.0,91.0,9.0,9.0,0.0,11.0,NaN,9.0,NaN,0.0,3.0,3.0,3.0,3.0,5.0,4.0,4.0,3.0,7.0,0.0,0.0,0.0,2.0,100.0,66.7,0.0,0.0,88635.0,55387.0,12500.0,75635.0
2,68466916,73356753.0,25000.0,25000.0,25000.0,36 months,7.49%,777.55,A,A4,Sales Manager,10+ years,MORTGAGE,109000.0,Not Verified,Dec-2015,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,debt_consolidation,Debt consolidation,226xx,VA,26.02,0.0,Dec-2001,1.0,NaN,NaN,9.0,0.0,20862.0,54.3%,19.0,w,0.00,0.00,26224.23,26224.23,25000.00,1224.23,0.0,0.0,0.0,Sep-2016,20807.39,NaN,Jan-2017,0.0,NaN,1.0,INDIVIDUAL,NaN,NaN,NaN,0.0,0.0,305781.0,0.0,3.0,0.0,1.0,13.0,47194.0,58.0,0.0,1.0,8937.0,57.0,38400.0,1.0,0.0,1.0,2.0,33976.0,17538.0,54.3,0.0,0.0,142.0,168.0,13.0,13.0,3.0,13.0,NaN,0.0,NaN,0.0,3.0,3.0,5.0,6.0,7.0,5.0,9.0,3.0,9.0,0.0,0.0,0.0,0.0,100.0,20.0,0.0,0.0,373572.0,68056.0,38400.0,82117.0
3,68466961,73356799.0,28000.0,28000.0,28000.0,36 months,6.49%,858.05,A,A2,Senior Manager,10+ years,MORTGAGE,92000.0,Not Verified,Dec-2015,Current,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,debt_consolidation,Debt consolidation,275xx,NC,21.60,0.0,May-1984,0.0,42.0,NaN,16.0,0.0,51507.0,6

In [4]:
# Convert ID and Interest Rate to numeric.
y2015['id'] = pd.to_numeric(y2015['id'], errors='coerce')
y2015['int_rate'] = pd.to_numeric(y2015['int_rate'].str.strip('%'), errors='coerce')

# Drop other columns with many unique variables
y2015.drop(['url', 'emp_title', 'zip_code', 'earliest_cr_line', 'revol_util',
            'sub_grade', 'addr_state', 'desc'], 1, inplace=True)
y2015 = y2015[:-2]

In [5]:
y2015.shape

(421095, 103)

In [6]:
from sklearn import ensemble
from sklearn.model_selection import cross_val_score

rfc = ensemble.RandomForestClassifier(n_jobs=-1)
X = y2015.drop(['loan_status', 'id','member_id'], 1)
Y = y2015['loan_status']
X = pd.get_dummies(X)
X = X.dropna(axis=1)
rfc_fit = rfc.fit(X, Y)
cross_val_score(rfc, X, Y, cv=10)

array([0.98124006, 0.98048016, 0.98154876, 0.98185747, 0.97402042,
       0.97746379, 0.96808283, 0.98052673, 0.98033581, 0.98076284])

After creating a list of 20 important features we will attempt to predict at a rate of atleast 90%

In [7]:
# Creating our list of 20 features
X2 = y2015[['annual_inc',
'delinq_2yrs',
'dti',
'emp_length',
'funded_amnt',
'funded_amnt_inv',
'grade',
'inq_last_6mths',
'installment',
'int_rate',
'last_pymnt_amnt',
'loan_amnt',
'num_accts_ever_120_pd',
'num_il_tl',
'num_op_rev_tl',
'num_rev_accts',
'num_sats',
'open_acc',
'out_prncp_inv',
'purpose'
           ]]
Y2 = y2015['loan_status']

X2 = pd.get_dummies(X2)
X2 = X2.dropna(axis=1)
rfc_fit = rfc.fit(X2, Y2)
cross_val_score(rfc, X2, Y2, cv=10)

array([0.94903944, 0.95181781, 0.95034552, 0.95215027, 0.951128  ,
       0.9502256 , 0.9504144 , 0.95067563, 0.95015081, 0.95069586])

It looks like it was a success predicting well over 90% accuracy with just 20 features. Let's try it again after removing last_pymnt_amnt and out_prncp_inv since they are related to payment amount and outstanding principals.

In [8]:
# Removing last_pymnt_amnt and out_prncp_inv
X3 = y2015[['annual_inc',
'delinq_2yrs',
'dti',
'emp_length',
'funded_amnt',
'funded_amnt_inv',
'grade',
'inq_last_6mths',
'installment',
'int_rate',
#'last_pymnt_amnt',
'loan_amnt',
'num_accts_ever_120_pd',
'num_il_tl',
'num_op_rev_tl',
'num_rev_accts',
'num_sats',
'open_acc',
#'out_prncp_inv'
'purpose'
           ]]
Y3 = y2015['loan_status']

X3 = pd.get_dummies(X3)
X3 = X2.dropna(axis=1)
rfc_fit = rfc.fit(X3, Y3)
cross_val_score(rfc, X3, Y3, cv=10)

array([0.9489682 , 0.95238774, 0.94958562, 0.95141412, 0.95174543,
       0.95024935, 0.95077062, 0.95067563, 0.9490821 , 0.94957963])

We are able to maintain a high prediction rate without the aforementioned variables. Now let's take a look at how valuable they are as it didnt change our cross validation score much

In [9]:
X4 = y2015[[#'annual_inc',
# 'delinq_2yrs',
# 'dti',
# 'emp_length',
# 'funded_amnt',
# 'funded_amnt_inv',
# 'grade',
# 'inq_last_6mths',
# 'installment',
# 'int_rate',
'last_pymnt_amnt',
# 'loan_amnt',
# 'num_accts_ever_120_pd',
# 'num_il_tl',
# 'num_op_rev_tl',
# 'num_rev_accts',
# 'num_sats',
# 'open_acc',
'out_prncp_inv'
#'purpose'
           ]]
Y4 = y2015['loan_status']

X4 = pd.get_dummies(X4)
X4 = X2.dropna(axis=1)
rfc_fit = rfc.fit(X4, Y4)
cross_val_score(rfc, X4, Y4, cv=10)

array([0.95039301, 0.95283893, 0.95013179, 0.95110541, 0.951128  ,
       0.94944194, 0.95134057, 0.95093685, 0.95076828, 0.9483684 ])

'last_pymnt_amnt' and 'out_prncp_inv' are important features capable of offering a prediction rate of 95% combined, but what about by themselves?

In [10]:
X5 = y2015[[#'annual_inc',
# 'delinq_2yrs',
# 'dti',
# 'emp_length',
# 'funded_amnt',
# 'funded_amnt_inv',
# 'grade',
# 'inq_last_6mths',
# 'installment',
# 'int_rate',
#'last_pymnt_amnt',
# 'loan_amnt',
# 'num_accts_ever_120_pd',
# 'num_il_tl',
# 'num_op_rev_tl',
# 'num_rev_accts',
# 'num_sats',
# 'open_acc',
'out_prncp_inv'
#'purpose'
           ]]
Y5 = y2015['loan_status']

X5 = pd.get_dummies(X5)
X5 = X2.dropna(axis=1)
rfc_fit = rfc.fit(X5, Y5)
cross_val_score(rfc, X5, Y5, cv=10)

array([0.94930066, 0.9526727 , 0.95015554, 0.95172283, 0.95174543,
       0.94994063, 0.95179178, 0.95058064, 0.94977082, 0.94889089])

In [11]:
X6 = y2015[[#'annual_inc',
# 'delinq_2yrs',
# 'dti',
# 'emp_length',
# 'funded_amnt',
# 'funded_amnt_inv',
# 'grade',
# 'inq_last_6mths',
# 'installment',
# 'int_rate',
'last_pymnt_amnt',
# 'loan_amnt',
# 'num_accts_ever_120_pd',
# 'num_il_tl',
# 'num_op_rev_tl',
# 'num_rev_accts',
# 'num_sats',
# 'open_acc',
#'out_prncp_inv'
#'purpose'
           ]]
Y6 = y2015['loan_status']

X6 = pd.get_dummies(X6)
X5 = X2.dropna(axis=1)
rfc_fit = rfc.fit(X6, Y6)
cross_val_score(rfc, X6, Y6, cv=10)

array([0.84172781, 0.84837691, 0.86464344, 0.86552207, 0.86485395,
       0.86019948, 0.86439953, 0.85924624, 0.85695965, 0.67854938])

It appears that both are important with out_prncp_inv having a similar score to using both. last_pymnt_amnt is a little less valuable with an score of only about 85%